# Asynchronous Bayesian optimisation

In [1]:
import torch
from nubo.acquisition import MCExpectedImprovement, MCUpperConfidenceBound
from nubo.models import GaussianProcess, fit_gp
from nubo.optimisation import joint
from nubo.test_functions import Hartmann6D
from nubo.utils import gen_inputs
from gpytorch.likelihoods import GaussianLikelihood


# test function
func = Hartmann6D(minimise=False)
dims = func.dims
bounds = func.bounds

# training data
x_train = gen_inputs(num_points=dims*5,
                     num_dims=dims,
                     bounds=bounds)
y_train = func(x_train)

# point pending evaluation
x_pending = torch.rand((1, dims))
print(f"Point pending evaluation: {x_pending.numpy().reshape(dims).round(4)}")

# Bayesian optimisation loop
iters = 10

for iter in range(iters):
    
    # specify Gaussian process
    likelihood = GaussianLikelihood()
    gp = GaussianProcess(x_train, y_train, likelihood=likelihood)
    
    # fit Gaussian process
    fit_gp(x_train, y_train, gp=gp, likelihood=likelihood, lr=0.1, steps=200)

    # specify acquisition function
    # acq = MCExpectedImprovement(gp=gp, y_best=torch.max(y_train), x_pending=x_pending, samples=256)
    acq = MCUpperConfidenceBound(gp=gp, beta=1.96**2, x_pending=x_pending, samples=256)

    # optimise acquisition function
    x_new, _ = joint(func=acq, method="Adam", batch_size=4, bounds=bounds, lr=0.1, steps=200, num_starts=1)

    # evaluate new point
    y_new = func(x_new)
    
    # add to data
    x_train = torch.vstack((x_train, x_new))
    y_train = torch.hstack((y_train, y_new))

    # print new best
    if torch.max(y_new) > torch.max(y_train[:-y_new.size(0)]):
        best_eval = torch.argmax(y_train)
        print(f"New best at evaluation {best_eval+1}: \t Inputs: {x_train[best_eval, :].numpy().reshape(dims).round(4)}, \t Outputs: {-y_train[best_eval].numpy().round(4)}")

# results
best_iter = int(torch.argmax(y_train))
print(f"Evaluation: {best_iter+1} \t Solution: {float(y_train[best_iter]):.4f}")


Point pending evaluation: [0.6839 0.9712 0.6219 0.8499 0.501  0.9095]
New best at evaluation 51: 	 Inputs: [3.881e-01 6.274e-01 3.000e-04 4.243e-01 2.554e-01 3.000e-04], 	 Outputs: -1.4441
New best at evaluation 55: 	 Inputs: [3.719e-01 8.601e-01 3.000e-04 5.365e-01 2.241e-01 3.100e-03], 	 Outputs: -2.9207
New best at evaluation 59: 	 Inputs: [0.3963 0.9244 0.0608 0.5594 0.1415 0.0038], 	 Outputs: -2.9958
New best at evaluation 69: 	 Inputs: [0.4065 0.8994 0.9944 0.5574 0.1096 0.0139], 	 Outputs: -3.157
Evaluation: 69 	 Solution: 3.1570
